### STEP : Diversity Analysis



#### Example

- [“Moving Pictures” tutorial](https://docs.qiime2.org/2022.8/tutorials/moving-pictures-usage/?highlight=ancom)
- [Biostats ANCOM](http://scikit-bio.org/docs/0.4.2/generated/generated/skbio.stats.composition.ancom.html)


#### Methods
- [composition](https://docs.qiime2.org/2022.8/plugins/available/composition/)
- [composition add_pseudocount](https://docs.qiime2.org/2022.8/plugins/available/composition/add-pseudocount/): Increment all counts in table by pseudocount.
- [composition ancom](https://docs.qiime2.org/2022.8/plugins/available/composition/ancom/): Apply ANCOM to identify features that differ in abundance.

## Setup and settings

In [1]:
# Importing packages
import os
import pandas as pd
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata

from qiime2.plugins.composition.visualizers import ancom
from qiime2.plugins.composition.methods import add_pseudocount

from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.feature_table.methods import filter_seqs
from qiime2.plugins.feature_table.methods import relative_frequency

from qiime2.plugins.taxa.methods import collapse

import biom

%matplotlib inline

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
metadata_file = '/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/metadata/metadata-ana-flavia-NCxNR.tsv'
base_dir = os.path.join('/', 'home', 'lauro', 'nupeb', 'rede-micro', 'redemicro-miliane-nutri')
base_dir = '/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri'
experiment_name = 'miliane-CxAC-trim'
class_col = 'group-id'
replace_files = False

In [3]:
# Parameters
base_dir = "/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri"
class_col = "group-id"
classifier_file = "/mnt/nupeb/rede-micro/datasets/16S_classifiers_qiime2/silva138.1_AB_V3-V4_classifier.qza"
experiment_name = "ana-flavia-STD-NCxSTD-NRxHSD-NCxHSD-NR"
manifest_file = "/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/selected_groups/manifest_STD-NCxSTD-NRxHSD-NCxHSD-NR.csv"
metadata_file = "/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/selected_groups/metadata_STD-NCxSTD-NRxHSD-NCxHSD-NR.tsv"
overlap = 12
phred = 20
replace_files = False
threads = 6
top_n = 20
trim = {"forward_primer": "CCTACGGGRSGCAGCAG", "overlap": 8, "reverse_primer": "GGACTACHVGGGTWTCTAAT"}
trunc_f = 0
trunc_r = 0


In [4]:
experiment_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name))
img_folder = os.path.abspath(os.path.join(experiment_folder, 'imgs'))

### Defining names, paths and flags

In [5]:
# QIIME2 Artifacts folder
qiime_folder = os.path.join(experiment_folder, 'qiime-artifacts')

# Input - DADA2 Artifacts
dada2_tabs_path = os.path.join(qiime_folder, 'dada2-tabs.qza')

In [6]:
qiime_folder
!pwd

/home/lauro/rede-micro/redemicro-ana-flavia-nutri


## Step execution

### Load input files

This Step import the QIIME2 `FeatureTable[Frequency]` Artifact and the `Metadata` file.

In [7]:
metadata_file
!ls ../data/raw/metadata

ls: cannot access '../data/raw/metadata': No such file or directory


In [8]:
#Load Metadata
metadata_qa = Metadata.load(metadata_file)

#Load FeatureTable[Frequency]
tabs = Artifact.load(dada2_tabs_path)

# Filter FeatureTable[Frequency | RelativeFrequency | PresenceAbsence | Composition] based on Metadata sample ID values
tabs = filter_samples(
    table=tabs,
    metadata=metadata_qa,
).filtered_table

# ANCOM

Apply Analysis of Composition of Microbiomes (ANCOM) to identify features
that are differentially abundant across groups.

- [composition add_pseudocount](https://docs.qiime2.org/2022.8/plugins/available/composition/add-pseudocount/): Increment all counts in table by pseudocount.
- [composition ancom](https://docs.qiime2.org/2022.8/plugins/available/composition/ancom/): Apply ANCOM to identify features that differ in abundance.

In [9]:
# Select class column
column = metadata_qa.get_column(class_col)

In [10]:
# Create the pseudocount table
composition_tab = add_pseudocount(table = tabs).composition_table

In [11]:
# Create visualizations for each parameter combination for the ANCOM  method.
transform_functions = ('sqrt', 'log', 'clr')
difference_functions = ('mean_difference', 'f_statistic')
for t in transform_functions:
    for d in difference_functions:
        print(f"Calculating ANCOM with: {t} {d}")
        try:
            ancom_viz = ancom(
                table = composition_tab, 
                metadata = column, 
                transform_function = t, 
                difference_function = d
            ).visualization
            view_name = os.path.join(qiime_folder, f'ancom-{t}-{d}.qzv')
            ancom_viz.save(view_name)
        except Exception as e:
            print(f"ERROR: Calculating ANCOM with: {t} {d}")
            print(e)

Calculating ANCOM with: sqrt mean_difference


/opt/anaconda3/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)


ERROR: Calculating ANCOM with: sqrt mean_difference
<lambda>() takes 2 positional arguments but 4 were given
Calculating ANCOM with: sqrt f_statistic


/opt/anaconda3/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)


/opt/anaconda3/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_composition/_ancom.py:103: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


...

Calculating ANCOM with: clr mean_difference


/opt/anaconda3/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)


ERROR: Calculating ANCOM with: clr mean_difference
<lambda>() takes 2 positional arguments but 4 were given
Calculating ANCOM with: clr f_statistic


/opt/anaconda3/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)


/opt/anaconda3/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_composition/_ancom.py:103: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


# LEfSe

LEfSe (Linear discriminant analysis Effect Size) determines the features (organisms, clades, operational taxonomic units, genes, or functions) most likely to explain differences between classes by coupling standard tests for statistical significance with additional tests encoding biological consistency and effect relevance.

- https://huttenhower.sph.harvard.edu/lefse/
- https://github.com/statonlab/BiGG2020_CrackNAg/wiki/qiime2-to-lefse
- https://github.com/biobakery/biobakery/wiki/lefse#2-lefse--conda-docker-vm-